<a href="https://colab.research.google.com/github/rubanzasilva/mnist_basics/blob/main/a_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.6 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *
import numpy as np
import pandas as pd

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
#hide
Path.BASE_PATH = path

In [5]:
#read in training digits
zeros = (path/'training/0').ls().sorted()
ones = (path/'training/1').ls().sorted()
twos = (path/'training/2').ls().sorted()
threes = (path/'training/3').ls().sorted()
fours = (path/'training/4').ls().sorted()
fives = (path/'training/5').ls().sorted()
sixes = (path/'training/6').ls().sorted()
sevens = (path/'training/7').ls().sorted()
eights = (path/'training/8').ls().sorted()
nines = (path/'training/9').ls().sorted()

#read in test set digits
valid_zeros = (path/'testing/0').ls().sorted()
valid_ones = (path/'testing/1').ls().sorted()
valid_twos = (path/'testing/2').ls().sorted()
valid_threes = (path/'testing/3').ls().sorted()
valid_fours = (path/'testing/4').ls().sorted()
valid_fives = (path/'testing/5').ls().sorted()
valid_sixes = (path/'testing/6').ls().sorted()
valid_sevens = (path/'testing/7').ls().sorted()
valid_eights = (path/'testing/8').ls().sorted()
valid_nines = (path/'testing/9').ls().sorted()

#convert all to tensor
zeros_tens =  [tensor(Image.open(o)) for o in zeros]
ones_tens = [tensor(Image.open(o)) for o in ones]
twos_tens = [tensor(Image.open(o)) for o in twos]
threes_tens = [tensor(Image.open(o)) for o in threes]
fours_tens = [tensor(Image.open(o)) for o in fours]
fives_tens = [tensor(Image.open(o)) for o in fives]
sixes_tens = [tensor(Image.open(o)) for o in sixes]
sevens_tens = [tensor(Image.open(o)) for o in sevens]
eights_tens = [tensor(Image.open(o)) for o in eights]
nines_tens = [tensor(Image.open(o)) for o in nines]

#convert all to tensor
valid_zeros_tens =  [tensor(Image.open(o)) for o in valid_zeros]
valid_ones_tens = [tensor(Image.open(o)) for o in valid_ones]
valid_twos_tens = [tensor(Image.open(o)) for o in valid_twos]
valid_threes_tens = [tensor(Image.open(o)) for o in valid_threes]
valid_fours_tens = [tensor(Image.open(o)) for o in valid_fours]
valid_fives_tens = [tensor(Image.open(o)) for o in valid_fives]
valid_sixes_tens = [tensor(Image.open(o)) for o in valid_sixes]
valid_sevens_tens = [tensor(Image.open(o)) for o in valid_sevens]
valid_eights_tens = [tensor(Image.open(o)) for o in valid_eights]
valid_nines_tens = [tensor(Image.open(o)) for o in valid_nines]

stacked_zeros = torch.stack(zeros_tens).float()/255
stacked_ones = torch.stack(ones_tens).float()/255
stacked_twos = torch.stack(twos_tens).float()/255
stacked_threes = torch.stack(threes_tens).float()/255
stacked_fours = torch.stack(fours_tens).float()/255
stacked_fives = torch.stack(fives_tens).float()/255
stacked_sixes = torch.stack(sixes_tens).float()/255
stacked_sevens = torch.stack(sevens_tens).float()/255
stacked_eights = torch.stack(eights_tens).float()/255
stacked_nines = torch.stack(nines_tens).float()/255

valid_stacked_zeros = torch.stack(valid_zeros_tens).float()/255
valid_stacked_ones = torch.stack(valid_ones_tens).float()/255
valid_stacked_twos = torch.stack(valid_twos_tens).float()/255
valid_stacked_threes = torch.stack(valid_threes_tens).float()/255
valid_stacked_fours = torch.stack(valid_fours_tens).float()/255
valid_stacked_fives = torch.stack(valid_fives_tens).float()/255
valid_stacked_sixes = torch.stack(valid_sixes_tens).float()/255
valid_stacked_sevens = torch.stack(valid_sevens_tens).float()/255
valid_stacked_eights = torch.stack(valid_eights_tens).float()/255
valid_stacked_nines = torch.stack(valid_nines_tens).float()/255


In [6]:
train_x = torch.cat([stacked_zeros,stacked_ones,stacked_twos,stacked_threes,stacked_fours,stacked_fives,stacked_sixes,stacked_sevens,stacked_eights,stacked_nines]).view(-1, 28*28)
train_y = tensor( [0]*len(zeros) + [1]*len(ones) + [2]*len(twos) + [3]*len(threes) + [4]*len(fours) + [5]*len(fives) + [6]*len(sixes) + [7]*len(sevens) + [8]*len(eights) + [9]*len(nines)).unsqueeze(1)
dset = list(zip(train_x,train_y))
x,y = dset[0]
x.shape,y
valid_x = torch.cat([valid_stacked_zeros,valid_stacked_ones,valid_stacked_twos,valid_stacked_threes,valid_stacked_fours,valid_stacked_fives,valid_stacked_sixes,valid_stacked_sevens,valid_stacked_eights,valid_stacked_nines]).view(-1, 28*28)
valid_y = tensor( [0]*len(valid_zeros) + [1]*len(valid_ones) + [2]*len(valid_twos) + [3]*len(valid_threes) + [4]*len(valid_fours) + [5]*len(valid_fives) + [6]*len(valid_sixes) + [7]*len(valid_sevens) + [8]*len(valid_eights) + [9]*len(valid_nines)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

#define linear model
def linear1(xb):
  return xb@weights + bias

simple_net = nn.Sequential(
nn.Linear(28*28,50),
nn.ReLU(),
nn.Linear(50,10)
)

#define function to Initialize parameters and return random parameters
#requires_grad tells PyTorch to track gradients when init_params is used

def init_params(size,std=1.0):
  return (torch.randn(size)*std).requires_grad_()


weights = init_params((28*28,10))
bias = init_params(10)

dl = DataLoader(dset, batch_size=256)
xb,yb = first(dl)
xb.shape,yb.shape

valid_dl = DataLoader(valid_dset, batch_size=256)

#mini batch
batch = train_x[:4]
batch


label_batch = train_y[:4]
label_batch.shape

#using the train images as activations, then applying softmax to that
#batch_sm_acts = torch.softmax(batch,dim=1)
#batch_sm_acts

#idx = range(4)
#batch_sm_acts[idx, label_batch]



torch.Size([4, 1])

# Possible solution and theory

Issue might be with the way i input the train_x, and y basically the whole enxt cell as i get better results when i vary it

1. i can test by varying codecell b with other solutions and trying same F.cross_entropy loss

2. I can also test this by varying the loss function with other losses usued, including manual ones, the ones that result in the best accuracy possibly mean the loss is the best but best of all it gives me a way of testing a manual implementation of cross entropy loss.

In [14]:
#codecell b

import numpy as np
path = untar_data(URLs.MNIST)
train_path = (path/'training').ls().sorted()
tensors = [(tensor(Image.open(o)), int(path.name)) for path in train_path for o in path.ls()]
np.random.shuffle(tensors)
index = math.floor(len(tensors) * 0.8)
train, valid = tensors[:index], tensors[index:]
train_x = torch.cat([train[i][0].view(-1, 784).float()/255 for i in range(len(train))])
train_y = tensor([train[i][1] for i in range(len(train))])
dset = list(zip(train_x, train_y))
valid_x = torch.cat([valid[i][0].view(-1, 784).float()/255 for i in range(len(valid))])
valid_y = tensor([valid[i][1] for i in range(len(valid))])
valid_dset = list(zip(valid_x, valid_y))

In [19]:
def init_params(size,std=1.0):
  return (torch.randn(size)*std).requires_grad_()


weights = init_params((28*28,10))
bias = init_params(10)

dl = DataLoader(dset, batch_size=256)
xb,yb = first(dl)
xb.shape,yb.shape

valid_dl = DataLoader(valid_dset, batch_size=256)

#mini batch
batch = train_x[:4]
batch


label_batch = train_y[:4]
label_batch.shape

#using the train images as activations, then applying softmax to that
#batch_sm_acts = torch.softmax(batch,dim=1)
#batch_sm_acts


torch.Size([4])

In [24]:
linear_model = nn.Linear(28*28,10)

In [20]:
simple_net = nn.Sequential(
nn.Linear(28*28,50),
nn.ReLU(),
nn.Linear(50,10)
)

In [21]:
dls = DataLoaders(dl, valid_dl)

In [22]:
learner = Learner(dls,simple_net,opt_func=SGD,loss_func=nn.CrossEntropyLoss(),metrics=accuracy,lr=0.001)

In [25]:
learner_2 = Learner(dls,linear_model,opt_func=SGD,loss_func=nn.CrossEntropyLoss(),metrics=accuracy,lr=0.001)

In [23]:
learner.fit(10,0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.542238,0.430129,0.884250,00:01
1,0.364565,0.347575,0.900000,00:01
2,0.319978,0.315356,0.908833,00:00
3,0.293829,0.293723,0.913750,00:00
4,0.273245,0.276094,0.919333,00:01
5,0.255489,0.260882,0.923833,00:01
6,0.239653,0.247537,0.928000,00:01
7,0.225702,0.236160,0.931083,00:01
8,0.213409,0.225967,0.934167,00:00
9,0.202455,0.216897,0.936750,00:00


In [26]:
learner_2.fit(10,0.1)

/usr/local/lib/python3.10/dist-packages/fastai/torch_core.py:263: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  return getattr(torch, 'has_mps', False)


epoch,train_loss,valid_loss,accuracy,time
0,0.574332,0.499882,0.875167,00:00
1,0.439067,0.420621,0.886667,00:00
2,0.395441,0.387527,0.893500,00:00
3,0.372059,0.368324,0.897917,00:00
4,0.356737,0.355431,0.902083,00:00
5,0.345642,0.346028,0.903833,00:00
6,0.337105,0.338791,0.905333,00:00
7,0.330261,0.333009,0.906667,00:00
8,0.324605,0.328261,0.907167,00:01
9,0.319825,0.324277,0.907917,00:01
